In [ ]:
!pip install polygon-api-client

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import datetime
import pandas as pd
import numpy as np
from polygon import RESTClient
import glob

## Timeseries Data

In [ ]:
crypto_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRxA6a1mj2xkdNYbzQYbhijAB6yB_ijHO51IpemYqb4k-YVqOzAfTi35xoSECzlCXbvmy8Uf7RsCGBr/pub?gid=2120570852&single=true&output=csv')
crypto_data['Ticker'] = ['X:'+crypto_data['Ticker'][i] + 'USD' for i in range(len(crypto_data['Ticker']))]
crypto_data.head()

,Rank,Ticker,Name
0,1,X:BTCUSD,bitcoin
1,2,X:ETHUSD,ethereum
2,4,X:BNBUSD,bnb
3,6,X:ADAUSD,cardano
4,7,X:SOLUSD,solana


In [ ]:
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')


def main():
    #REPLACE KEY
    key = "ayaS0IEAdO4NbNiftfNf8jhNwKfoRJhQ"
    folderName = '/content/drive/Shareddrives/BTP/CryptoData/'
    tickers = crypto_data['Ticker'][:160]
    no_results = []
    no_vw = []
    count = -1
    
    # RESTClient can be used as a context manager to facilitate closing the underlying http session
    # https://requests.readthedocs.io/en/master/user/advanced/#session-objects
    with RESTClient(key) as client:
        from_ = "2015-01-01"
        to = "2022-02-01"
        for tic in tickers:
          count+=1
          resp = client.crypto_aggregates(tic, 1, "day", from_, to, unadjusted=False)
          print(f"Daily aggregate for {count} : {resp.ticker} between {from_} and {to}.")
          dates = []
          opening = []
          closing = []
          highest = []
          lowest = []
          v = []
          vw = []
          n = []      
          if resp.resultsCount==0:
            print('No results.')
            no_results.append(tic)
          else:
            for result in resp.results:
                dt = ts_to_datetime(result["t"])
                dates.append(dt)
                opening.append(result['o'])
                highest.append(result['h'])
                lowest.append(result['l'])
                closing.append(result['c'])
                n.append(result['n'])
                v.append(result['v'])
                if 'vw' in result:
                  vw.append(result['vw'])
                else:
                  vw.append('')
                
            df = pd.DataFrame(np.transpose([dates,opening,closing,highest,lowest,n,v, vw]), columns = ['Date','O','C','H','L','N','V','VW'])
            df.to_excel(folderName + tic.split(':')[-1][:-3]+'.xlsx')
            print(tic,' completed.')
    return no_results

if __name__ == '__main__':
    no_results_list = main()

In [ ]:
count =0
for _ in glob.glob('/content/drive/Shareddrives/BTP/CryptoData/*.xlsx'):
  count+=1
count

100

In [ ]:
pd.DataFrame(no_results_list,columns=['Ticker']).to_excel('/content/drive/Shareddrives/BTP/no_result_list.xlsx')

## Tweepy

In [4]:
# Python Script to Extract tweets of a
# particular Hashtag using Tweepy and Pandas


# import modules
import pandas as pd
import tweepy


# function to display data of each tweet
def printtweetdata(n, ith_tweet):
	print()
	print(f"Tweet {n}:")
	print(f"Username:{ith_tweet[0]}")
	print(f"Tweet Text:{ith_tweet[1]}")
	print(f"Hashtags Used:{ith_tweet[-1]}")


# function to perform data extraction
def scrape(words, date_since, date_till, numtweet):
	
	# Creating DataFrame using pandas
	db = pd.DataFrame(columns=['username', 'text', 'retweeted','tweet_time', 'description(bio)', 'acc_created', 'friends', 'followers', 'retweet_count','likes_count','hashtags'])
	
	# We are using .Cursor() to search through twitter for the required tweets.
	# The number of tweets can be restricted using .items(number of tweets)
	for k in range(len(words)):
		tweets = tweepy.Cursor(api.search, q=words[k], lang="en",
							since=date_since, tweet_mode='extended').items(numtweet)
		
		# .Cursor() returns an iterable object. Each item in
		# the iterator has various attributes that you can access to
		# get information about each tweet
		list_tweets = [tweet for tweet in tweets]
		
		# Counter to maintain Tweet Count
		i = 1
		
		# we will iterate over each tweet in the list for extracting information about each tweet
		for tweet in list_tweets:
			username = tweet.user.screen_name
			hashtags = tweet.entities['hashtags']
			# print(tweet.created_at, tweet.user.description, tweet.user.created_at, tweet.user.friends_count, tweet.user.followers_count)
			
			# Retweets can be distinguished by a retweeted_status attribute,
			# in case it is an invalid reference, except block will be executed
			try:
				text = tweet.retweeted_status.full_text
				retweeted = 'True'
			except AttributeError:
				text = tweet.full_text
				retweeted = 'False'
			
			hashtext = list()
			for j in range(0, len(hashtags)):
				hashtext.append(hashtags[j]['text'])
		
		  
			
			# Here we are appending all the extracted information in the DataFrame
			ith_tweet = [username, text, retweeted, tweet.created_at, tweet.user.description, tweet.user.created_at, tweet.user.friends_count, tweet.user.followers_count, tweet.retweet_count, tweet.favorite_count, hashtext]
			
			db.loc[len(db)] = ith_tweet
			
			# Function call to print tweet data on screen
			# printtweetdata(i, ith_tweet)
			if i==int(numtweet/2):
				print('50% complete for ',words[k])
			elif i==numtweet:
				print('100% complete for ',words[k])
			i = i+1
			
	filename = 'scraped_tweets.csv'
	
	# we will save our database as a CSV file.
	db.to_csv(filename)


if __name__ == '__main__':
	
	#Pratvi credentials
  consumer_key = "pXfG7o1OfnoYaZglzT7jwmSR1"
  consumer_secret = "Ez9AbZsV2kRfKJRJpYUOmvwXlUc1MTztgEtGmyoKcDl1yDhp1g"
  access_key = "1482958101964025856-jUZJxrNboZyeq6RKVfeM4yJpSTpwhB"
  access_secret = "bspdNWX1laW2dpudQdMvK84sgfWj9NfiA8QbydGXcFVpA"

	#Vishwa credentials
	# consumer_key = "jVcRthi6uPGz07Fp255MAHfsB"
	# consumer_secret = "1jc7zeAjsWP0ecQ8to6aDkkdBr09271MihwQh69OSTmT5WdWmb"
	# access_key = "1059738121263243265-Oy5KNZBmpaUm0GMbW5fPmVy398y8oJ"
	# access_secret = "sPK9gvoyLARwW490X5blEUKImEYaNmeaIQ6yADz3muTuY"
	
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

	# Enter Hashtag and initial date
  print("Enter Twitter HashTag to search for")
  words = ["#crypto"]
  print("Enter Date since The Tweets are required in yyyy-mm--dd")
  date_since = "2017-01-01";date_till = "2022-01-01";
	
	# number of tweets you want to extract in one run
  numtweet = 1
  scrape(words, date_since,date_till, numtweet)
  print('Scraping has completed!')


Enter Twitter HashTag to search for
Enter Date since The Tweets are required in yyyy-mm--dd
100% complete for  $btc
Scraping has completed!
